In [139]:
import pandas as pd

In [140]:

custom_column_names = ["instance", "feature", "value"]
df_train_x = pd.read_csv("train_X.csv",names=custom_column_names)
# merged_data_encoded = pd.get_dummies(df_train_x, columns=['feature'])
df_train_x.loc[df_train_x['instance'] ==1]
# merged_data_encoded[merged_data_encoded['instance'] ==5]

,instance,feature,value
4468,1,6,1
10448,1,9,1
19800,1,15,1
29120,1,22,1
39212,1,29,1
40830,1,33,1
48744,1,34,1
62480,1,37,1
68300,1,42,1
73116,1,50,1


In [141]:
custom_features_names = ["feature","value"]
df_features = pd.read_csv("feature_names.csv",sep="=",names=custom_features_names)
df_features

,feature,value
0,cap-shape,bell
1,cap-shape,conical
2,cap-shape,flat
3,cap-shape,knobbed
4,cap-shape,sunken
...,...,...
107,habitat,leaves
108,habitat,meadows
109,habitat,paths
110,habitat,urban


In [142]:
df_test_x = pd.read_csv("test_X.csv",names=custom_column_names)
df_test_x

,instance,feature,value
0,10,1,1
1,39,1,1
2,44,1,1
3,52,1,1
4,59,1,1
...,...,...,...
3995,186,111,1
3996,200,111,1
3997,16,112,1
3998,81,112,1


In [143]:
custom_Y_names = ["poisonous"]
df_test_Y = pd.read_csv("test_Y.csv",names=custom_Y_names)
df_test_Y

,poisonous
0,-1
1,1
2,-1
3,-1
4,1
...,...
195,1
196,-1
197,-1
198,1


In [144]:
df_pivot = df_train_x.pivot(index="instance", columns="feature", values="value").fillna(0)

# Reset the index to make "instance" a regular column
df_pivot.reset_index(inplace=True)

# Rename the columns for clarity (0 and 1 instead of 0.0 and 1.0)
df_pivot.columns = [f"feature_{col}" if col != "instance" else col for col in df_pivot.columns]

# Optional: Convert "instance" column to an integer (if it's not already)
df_pivot["instance"] = df_pivot["instance"]
df_pivot

,instance,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112
0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,8120,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8120,8121,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8121,8122,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8122,8123,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [145]:

df_train_Y = pd.read_csv("train_Y.csv",names=custom_Y_names)
df_train_Y

,poisonous
0,-1
1,1
2,1
3,-1
4,1
...,...
8119,1
8120,1
8121,1
8122,-1


In [146]:
df_test_x = pd.read_csv("test_X.csv",names=custom_column_names)
df_pivot_test = df_test_x.pivot(index="instance", columns="feature", values="value").fillna(0)

# Reset the index to make "instance" a regular column
df_pivot_test.reset_index(inplace=True)

# Rename the columns for clarity (0 and 1 instead of 0.0 and 1.0)
df_pivot_test.columns = [f"feature_{col}" if col != "instance" else col for col in df_pivot_test.columns]

# Optional: Convert "instance" column to an integer (if it's not already)
df_pivot_test["instance"] = df_pivot_test["instance"]
df_pivot_test

,instance,feature_1,feature_3,feature_4,feature_5,feature_6,feature_7,feature_9,feature_10,feature_11,...,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,197,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
197,198,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
198,199,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
